In [3]:
import pandas as pd
import numpy as np
from sklearn import linear_model
from sklearn.model_selection import cross_val_score
import torch
from torch import optim
import torch.nn as nn
from torch.autograd import Variable as V
import torch.nn.functional as F
from torch.utils.data import TensorDataset
from torch.utils.data import DataLoader

from sklearn.metrics import accuracy_score

team_dic = {"SAS":"San Antonio Spurs","SAC":"Sacramento Kings","UTA":"Utah Jazz",
                "DAL":"Dallas Mavericks","POR":"Portland Trail Blazers","LAL":"Los Angeles Lakers",
                "PHI":"Philadelphia 76ers","MIL":"Milwaukee Bucks","HOU":"Houston Rockets",
                "PHO":"Phoenix Suns","NYK":"New York Knicks","MIN":"Minnesota Timberwolves",
                "MIA":"Miami Heat","TOR":"Toronto Raptors",
                "CHH":"Charlotte Hornets",  #02-03 New Orleans Hornets
                "NOH":"New Orleans Hornets",
                "NOP":"New Orleans Pelicans",#13-14 New Orleans Pelicans
                "SEA":"Seattle SuperSonics",  
                "OKC":"Oklahoma City Thunder",#08-09 Oklahoma City Thunder
                "ORL":"Orlando Magic","IND":"Indiana Pacers","DEN":"Denver Nuggets",
                "DET":"Detroit Pistons","LAC":"Los Angeles Clippers","BOS":"Boston Celtics",
                "CLE":"Cleveland Cavaliers",   
                "VAN":"Vancouver Grizzlies",#1-2 Memphis Grizzlies
                "MEM":"Memphis Grizzlies",
                "NJN":"New Jersey Nets",
                "BRK":"Brooklyn Nets",#12-13 Brooklyn Nets
                "ATL":"Atlanta Hawks","WAS":"Washington Wizards",
                 "GSW":"Golden State Warriors",
                "CHI":"Chicago Bulls" ,
                "CHA":"Charlotte Bobcats",   #04-05 29-30 CHA Charlotte Bobcats
                "CHO":"Charlotte Hornets"  #14-15 Charlotte Hornets
           }

player_stat_00_01 = pd.read_csv("play_per_game_stat/00-01.csv",delimiter = ",")        
player_stat_01_02 = pd.read_csv("play_per_game_stat/01-02.csv",delimiter = ",")
player_stat_02_03 = pd.read_csv("play_per_game_stat/02-03.csv",delimiter = ",")
player_stat_03_04 = pd.read_csv("play_per_game_stat/03-04.csv",delimiter = ",")
# print(player_stat_03_04.head())
player_stat_04_05 = pd.read_csv("play_per_game_stat/04-05.csv",delimiter = ",")
player_stat_05_06 = pd.read_csv("play_per_game_stat/05-06.csv",delimiter = ",")
player_stat_06_07 = pd.read_csv("play_per_game_stat/06-07.csv",delimiter = ",")
player_stat_07_08 = pd.read_csv("play_per_game_stat/07-08.csv",delimiter = ",")
player_stat_08_09 = pd.read_csv("play_per_game_stat/08-09.csv",delimiter = ",")
player_stat_09_10 = pd.read_csv("play_per_game_stat/09-10.csv",delimiter = ",")

player_stat_10_11 = pd.read_csv("play_per_game_stat/10-11.csv",delimiter = ",")
player_stat_11_12 = pd.read_csv("play_per_game_stat/11-12.csv",delimiter = ",")
player_stat_12_13 = pd.read_csv("play_per_game_stat/12-13.csv",delimiter = ",")
player_stat_13_14 = pd.read_csv("play_per_game_stat/13-14.csv",delimiter = ",")
player_stat_14_15 = pd.read_csv("play_per_game_stat/14-15.csv",delimiter = ",")
player_stat_15_16 = pd.read_csv("play_per_game_stat/15-16.csv",delimiter = ",")
player_stat_16_17 = pd.read_csv("play_per_game_stat/16-17.csv",delimiter = ",")

stat_00_10 = [player_stat_00_01,player_stat_01_02,player_stat_02_03,player_stat_03_04,player_stat_04_05,
             player_stat_05_06,player_stat_06_07,player_stat_07_08,player_stat_08_09,player_stat_09_10]
              
stat_10_17 = [ player_stat_10_11,player_stat_11_12,player_stat_12_13,
              player_stat_13_14,player_stat_14_15,player_stat_15_16,player_stat_16_17] 


mvp_train_dataframe = pd.read_csv("mvp/mvp_train.csv",delimiter = ",")
mvp_test_dataframe = pd.read_csv("mvp/mvp_test.csv",delimiter = ",")


def initialize_team_miscellaneous_stat(season_begin, season_end , team_dic):
    a = []
    for i in np.arange(season_begin,season_end,1):
        miscellaneous_dataframe = pd.read_csv("miscellaneous_stat/"+str(i)+"-"+str(i+1)+".csv",delimiter = ",")
        miscellaneous_dataframe.fillna(0,inplace = True)
        column_num = miscellaneous_dataframe.shape[0]
        for j in range(column_num):
            if(miscellaneous_dataframe.loc[j,"Team"].find("*") != -1):
                miscellaneous_dataframe.loc[j,"Team"] = miscellaneous_dataframe.loc[j,"Team"][0:miscellaneous_dataframe.loc[j,"Team"].find("*")]
            for key in team_dic:
                if(team_dic[key] == miscellaneous_dataframe.loc[j,"Team"]):
                    miscellaneous_dataframe.loc[j,"Team"] = key
                    if((season_begin == 0) & (team_dic[key] == "Charlotte Hornets")):
                        miscellaneous_dataframe.loc[j,"Team"] = "CHH"
                    if((season_begin == 10) & (team_dic[key] == "Charlotte Hornets")):
                        if(i >=14):
                            miscellaneous_dataframe.loc[j,"Team"] = "CHO"     
        a.append(miscellaneous_dataframe)
    return a

miscellaneous_stat_train  = initialize_team_miscellaneous_stat(0,10,team_dic)
miscellaneous_stat_test = initialize_team_miscellaneous_stat(10,17,team_dic)
# print(miscellaneous_stat_train[0].shape)
# print(miscellaneous_stat_train[3].shape)
# print(miscellaneous_stat_train[4].shape)

def initialize_data(stat,mvp_dataframe,team_dic,team_miscellaneous_stat):
    a=[]
    b=[]
    d=[]
    mvp_list = []

    for i in range(len(stat)):
        a.append(stat[i])
        origin_column_num = a[i].shape[0]
        a[i]["W"] = 0 #新的一列
        for o in range(origin_column_num):
            team_abbreviation = stat[i].loc[o,"Tm"]
            for key in team_dic:
                if(team_abbreviation == key):
                    # series （6:56) --->list --->get value
                    serie = team_miscellaneous_stat[i].loc[team_miscellaneous_stat[i]["Team"]==key,"W"]
                    team_win_num = serie.values[0]
                    a[i].loc[o,"W"] = team_win_num
        a[i] = a[i].drop(["Pos","Tm","Rk"],axis=1)            
        a[i].sort_values("PTS",ascending=False, inplace=True)
        a[i]= a[i].reset_index(drop=True)  # 来改变index就可以了,
        a[i] = a[i][0:50]
        a[i].fillna(0,inplace = True)

        column_num = a[i].shape[0] #50
        c = []
        for j in range(column_num):     
            if(a[i].loc[j,"Player"].find("*") != -1):
                a[i].loc[j,"Player"] = a[i].loc[j,"Player"][0:a[i].loc[j,"Player"].find("*")]
            if(a[i].loc[j,"Player"].find("\\") != -1):
                a[i].loc[j,"Player"] = a[i].loc[j,"Player"][0:a[i].loc[j,"Player"].find("\\")]          
            if (a[i].loc[j,"Player"] == mvp_dataframe.loc[i,"name"]):
                mvp_list.append(j)
            print(a[i].info())
            c.append(list(a[i].loc[j,a[i].columns.difference(["Player"])]))
        b.append(c)
        d.append(list(a[i]["Player"]))
    return (d,mvp_list,b)

name_list_train, mvp_list_train, X_train_list = initialize_data(stat_00_10,mvp_train_dataframe,team_dic,miscellaneous_stat_train)
print("|"*50)
name_list_test, mvp_list_test ,X_test_list= initialize_data(stat_10_17,mvp_test_dataframe,team_dic,miscellaneous_stat_test)


print(len(X_train_list[0][0]))

# for k in range(10):
#     print(name_list_train[k][mvp_list_train[k]])

# print(mvp_list_test)
# for k in range(7):
#     print(name_list_test[k][mvp_list_test[k]])



<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50 entries, 0 to 49
Data columns (total 28 columns):
Player    50 non-null object
Age       50 non-null int64
G         50 non-null int64
GS        50 non-null int64
MP        50 non-null float64
FG        50 non-null float64
FGA       50 non-null float64
FG%       50 non-null float64
3P        50 non-null float64
3PA       50 non-null float64
3P%       50 non-null float64
2P        50 non-null float64
2PA       50 non-null float64
2P%       50 non-null float64
eFG%      50 non-null float64
FT        50 non-null float64
FTA       50 non-null float64
FT%       50 non-null float64
ORB       50 non-null float64
DRB       50 non-null float64
TRB       50 non-null float64
AST       50 non-null float64
STL       50 non-null float64
BLK       50 non-null float64
TOV       50 non-null float64
PF        50 non-null float64
PTS       50 non-null float64
W         50 non-null int64
dtypes: float64(23), int64(4), object(1)
memory usage: 11.1+ KB
No

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50 entries, 0 to 49
Data columns (total 28 columns):
Player    50 non-null object
Age       50 non-null int64
G         50 non-null int64
GS        50 non-null int64
MP        50 non-null float64
FG        50 non-null float64
FGA       50 non-null float64
FG%       50 non-null float64
3P        50 non-null float64
3PA       50 non-null float64
3P%       50 non-null float64
2P        50 non-null float64
2PA       50 non-null float64
2P%       50 non-null float64
eFG%      50 non-null float64
FT        50 non-null float64
FTA       50 non-null float64
FT%       50 non-null float64
ORB       50 non-null float64
DRB       50 non-null float64
TRB       50 non-null float64
AST       50 non-null float64
STL       50 non-null float64
BLK       50 non-null float64
TOV       50 non-null float64
PF        50 non-null float64
PTS       50 non-null float64
W         50 non-null int64
dtypes: float64(23), int64(4), object(1)
memory usage: 11.1+ KB
No

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50 entries, 0 to 49
Data columns (total 28 columns):
Player    50 non-null object
Age       50 non-null int64
G         50 non-null int64
GS        50 non-null int64
MP        50 non-null float64
FG        50 non-null float64
FGA       50 non-null float64
FG%       50 non-null float64
3P        50 non-null float64
3PA       50 non-null float64
3P%       50 non-null float64
2P        50 non-null float64
2PA       50 non-null float64
2P%       50 non-null float64
eFG%      50 non-null float64
FT        50 non-null float64
FTA       50 non-null float64
FT%       50 non-null float64
ORB       50 non-null float64
DRB       50 non-null float64
TRB       50 non-null float64
AST       50 non-null float64
STL       50 non-null float64
BLK       50 non-null float64
TOV       50 non-null float64
PF        50 non-null float64
PTS       50 non-null float64
W         50 non-null int64
dtypes: float64(23), int64(4), object(1)
memory usage: 11.1+ KB
No

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50 entries, 0 to 49
Data columns (total 28 columns):
Player    50 non-null object
Age       50 non-null int64
G         50 non-null int64
GS        50 non-null int64
MP        50 non-null float64
FG        50 non-null float64
FGA       50 non-null float64
FG%       50 non-null float64
3P        50 non-null float64
3PA       50 non-null float64
3P%       50 non-null float64
2P        50 non-null float64
2PA       50 non-null float64
2P%       50 non-null float64
eFG%      50 non-null float64
FT        50 non-null float64
FTA       50 non-null float64
FT%       50 non-null float64
ORB       50 non-null float64
DRB       50 non-null float64
TRB       50 non-null float64
AST       50 non-null float64
STL       50 non-null float64
BLK       50 non-null float64
TOV       50 non-null float64
PF        50 non-null float64
PTS       50 non-null float64
W         50 non-null int64
dtypes: float64(23), int64(4), object(1)
memory usage: 11.1+ KB
No

None
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50 entries, 0 to 49
Data columns (total 28 columns):
Player    50 non-null object
Age       50 non-null int64
G         50 non-null int64
GS        50 non-null int64
MP        50 non-null float64
FG        50 non-null float64
FGA       50 non-null float64
FG%       50 non-null float64
3P        50 non-null float64
3PA       50 non-null float64
3P%       50 non-null float64
2P        50 non-null float64
2PA       50 non-null float64
2P%       50 non-null float64
eFG%      50 non-null float64
FT        50 non-null float64
FTA       50 non-null float64
FT%       50 non-null float64
ORB       50 non-null float64
DRB       50 non-null float64
TRB       50 non-null float64
AST       50 non-null float64
STL       50 non-null float64
BLK       50 non-null float64
TOV       50 non-null float64
PF        50 non-null float64
PTS       50 non-null float64
W         50 non-null int64
dtypes: float64(23), int64(4), object(1)
memory usage: 11.1+ 

...

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50 entries, 0 to 49
Data columns (total 28 columns):
Player    50 non-null object
Age       50 non-null int64
G         50 non-null int64
GS        50 non-null int64
MP        50 non-null float64
FG        50 non-null float64
FGA       50 non-null float64
FG%       50 non-null float64
3P        50 non-null float64
3PA       50 non-null float64
3P%       50 non-null float64
2P        50 non-null float64
2PA       50 non-null float64
2P%       50 non-null float64
eFG%      50 non-null float64
FT        50 non-null float64
FTA       50 non-null float64
FT%       50 non-null float64
ORB       50 non-null float64
DRB       50 non-null float64
TRB       50 non-null float64
AST       50 non-null float64
STL       50 non-null float64
BLK       50 non-null float64
TOV       50 non-null float64
PF        50 non-null float64
PTS       50 non-null float64
W         50 non-null int64
dtypes: float64(23), int64(4), object(1)
memory usage: 11.1+ KB
No

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50 entries, 0 to 49
Data columns (total 28 columns):
Player    50 non-null object
Age       50 non-null int64
G         50 non-null int64
GS        50 non-null int64
MP        50 non-null float64
FG        50 non-null float64
FGA       50 non-null float64
FG%       50 non-null float64
3P        50 non-null float64
3PA       50 non-null float64
3P%       50 non-null float64
2P        50 non-null float64
2PA       50 non-null float64
2P%       50 non-null float64
eFG%      50 non-null float64
FT        50 non-null float64
FTA       50 non-null float64
FT%       50 non-null float64
ORB       50 non-null float64
DRB       50 non-null float64
TRB       50 non-null float64
AST       50 non-null float64
STL       50 non-null float64
BLK       50 non-null float64
TOV       50 non-null float64
PF        50 non-null float64
PTS       50 non-null float64
W         50 non-null int64
dtypes: float64(23), int64(4), object(1)
memory usage: 11.1+ KB
No

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50 entries, 0 to 49
Data columns (total 28 columns):
Player    50 non-null object
Age       50 non-null int64
G         50 non-null int64
GS        50 non-null int64
MP        50 non-null float64
FG        50 non-null float64
FGA       50 non-null float64
FG%       50 non-null float64
3P        50 non-null float64
3PA       50 non-null float64
3P%       50 non-null float64
2P        50 non-null float64
2PA       50 non-null float64
2P%       50 non-null float64
eFG%      50 non-null float64
FT        50 non-null float64
FTA       50 non-null float64
FT%       50 non-null float64
ORB       50 non-null float64
DRB       50 non-null float64
TRB       50 non-null float64
AST       50 non-null float64
STL       50 non-null float64
BLK       50 non-null float64
TOV       50 non-null float64
PF        50 non-null float64
PTS       50 non-null float64
W         50 non-null int64
dtypes: float64(23), int64(4), object(1)
memory usage: 11.1+ KB
No

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50 entries, 0 to 49
Data columns (total 28 columns):
Player    50 non-null object
Age       50 non-null int64
G         50 non-null int64
GS        50 non-null int64
MP        50 non-null float64
FG        50 non-null float64
FGA       50 non-null float64
FG%       50 non-null float64
3P        50 non-null float64
3PA       50 non-null float64
3P%       50 non-null float64
2P        50 non-null float64
2PA       50 non-null float64
2P%       50 non-null float64
eFG%      50 non-null float64
FT        50 non-null float64
FTA       50 non-null float64
FT%       50 non-null float64
ORB       50 non-null float64
DRB       50 non-null float64
TRB       50 non-null float64
AST       50 non-null float64
STL       50 non-null float64
BLK       50 non-null float64
TOV       50 non-null float64
PF        50 non-null float64
PTS       50 non-null float64
W         50 non-null int64
dtypes: float64(23), int64(4), object(1)
memory usage: 11.1+ KB
No

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50 entries, 0 to 49
Data columns (total 28 columns):
Player    50 non-null object
Age       50 non-null int64
G         50 non-null int64
GS        50 non-null int64
MP        50 non-null float64
FG        50 non-null float64
FGA       50 non-null float64
FG%       50 non-null float64
3P        50 non-null float64
3PA       50 non-null float64
3P%       50 non-null float64
2P        50 non-null float64
2PA       50 non-null float64
2P%       50 non-null float64
eFG%      50 non-null float64
FT        50 non-null float64
FTA       50 non-null float64
FT%       50 non-null float64
ORB       50 non-null float64
DRB       50 non-null float64
TRB       50 non-null float64
AST       50 non-null float64
STL       50 non-null float64
BLK       50 non-null float64
TOV       50 non-null float64
PF        50 non-null float64
PTS       50 non-null float64
W         50 non-null int64
dtypes: float64(23), int64(4), object(1)
memory usage: 11.1+ KB
No

In [ ]:
X_train_array = np.array(X_train_list)
print(X_train_array.shape)
X_train_array = np.delete(X_train_array,4,axis = 0)
X_train_array = np.delete(X_train_array,5,axis = 0)
X_train = torch.tensor(X_train_array,dtype=torch.float).view(-1,50*27)

# X_train = torch.tensor(np.array(X_train_list),dtype=torch.float)
mvp_array_train = np.array(mvp_list_train)
mvp_array_train = np.delete(mvp_array_train,4,axis = 0)
mvp_array_train = np.delete(mvp_array_train,5,axis = 0)
y_train = torch.tensor(mvp_array_train,dtype=torch.long)

# X_test = torch.tensor(np.array(X_test_list),dtype=torch.float)
X_test = torch.tensor(np.array(X_test_list),dtype=torch.float).view(-1,50*27)
y_test = torch.tensor(mvp_list_test,dtype=torch.long)

print(X_train.shape)
print(y_train.shape)
print("*")
print(X_test.shape)
print(y_test.shape)
print(X_train.dtype)
print(y_train.dtype)
print(X_test.dtype)
print(y_test.dtype)

print(y_train)
print(y_test)
class Model(torch.nn.Module): 
    def __init__(self):
        super(Model, self).__init__() 
        self.linear1 = torch.nn.Linear(1350, 600)    
        self.linear2 = torch.nn.Linear(600, 300) 
        self.linear3= torch.nn.Linear(300, 50)
        
    def forward(self, x):
        x = F.relu(self.linear1(x))
        x = F.relu(self.linear2(x))
        x = self.linear3(x)
        return x

def train_model(model, X_train, y_train, X_test, y_test, epochs=30, batch_size=3, lr=0.1, weight_decay=0):
#     train_dataset = TensorDataset(X_train, y_train)
#     train_data_loader = DataLoader(train_dataset, batch_size=batch_size)
    # loss function
#     loss_func = F.cross_entropy
    loss_func = torch.nn.CrossEntropyLoss()
    # optimizer
    optimizer = optim.SGD(model.parameters(), lr=lr, weight_decay=weight_decay)
    # training loop
    for epoch in range(epochs):
        pred = model(X_train)
        loss = loss_func(pred, y_train)
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()   
#         acc = accuracy_score(torch.argmax(y_pred, dim=1).detach().numpy(), y_test)
#         print('Loss at epoch %d : %f, test acc: %f' % (epoch, loss, acc))
#     for i in range(y_pred.shape[0]):
#     print(y_pred_softmax)
#     for item in y_pred_softmax:
#         print(item)
        
#     print(y_pred)    

model = Model()
train_model(model, X_train, y_train, X_test, y_test, lr=0.01)

y_pred = model(X_test)
y_pred_softmax = F.softmax(y_pred,dim=1)
print()
y_pred = y_pred_softmax.detach().numpy()

print(np.argmax(y_pred, axis=1),np.max(y_pred, axis=1)) 


